$Signal\;and\;system$

$Phase\;2$

In [225]:
import cv2
import pydub
import numpy as np
import sounddevice as sd
from matplotlib import pyplot as plt
from scipy.io.wavfile import read, write
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
from itertools import zip_longest

In [298]:
notes_base = 2**(np.arange(12)/12)*27.5
notes_duration = np.array([3200, 1600, 800, 400, 200, 100])*0.7
notes_ann = ['A', 'A#', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#']

def sin_wave(f, n, fs):
    x = np.linspace(0, 2*np.pi, n)
    ring = 30 
    xp = np.linspace(0, -1*(n*ring/fs), n)
    y = np.sin(x*f*(n/fs))*np.exp(xp)
    z = np.zeros([n, 2])
    z[:, 0] = y
    z[:, 1] = y
    return z

def play_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]*(fs/1000)), fs)
    sd.play(y, fs)
    sd.wait()
    return 

def put_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]*(fs/1000)), fs)
    return y

def get_music(music_notes, fs):
    m = []
    for item in music_notes:
        y = put_note(item[0], item[1], item[2], fs)
        m.append(y)
    m = np.concatenate(m, 0)
    return m

fs1 = 44100
music = [[8, 5, 3], [10, 5, 4], [10, 5, 4], [10, 5, 3], [10, 5, 3], [10, 5, 3], [10, 5, 3], [10, 5, 3], [10, 5, 3], 
        [11, 5, 4], [4, 5, 4], [11, 5, 4], [4, 5, 4], [11, 5, 4], [4, 5, 4], [11, 5, 4], [4, 5, 4], [3, 5, 3], 
        [8, 5, 4], [8, 5, 4], [8, 5, 3], [8, 5, 3], [8, 5, 3], [8, 5, 3], [8, 5, 3], [8, 5, 3], 
        [10, 5, 4], [3, 5, 4], [10, 5, 4], [3, 5, 4], [10, 5, 4], [3, 5, 4], [10, 5, 4], [3, 5, 4], [1, 4, 3]]

y = get_music(music, fs1)
sd.play(y, fs1)

In [299]:
Scale = [[3,4,3], [5,4,3], [7,4,3], [8,4,3], [10,4,3], [0,4,3], [2,4,3], [3,5,3], 
        [2,4,3], [0,4,3], [10,4,3], [8,4,3], [7,4,3], [5,4,3], [3,4,3]]



$Matching\;Template$

In [179]:
template_path = 'template.jpg'
background_path = 'background.jpg'

template = cv2.imread(template_path)
background = cv2.imread(background_path)

template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

In [180]:
result = cv2.matchTemplate(background_gray, template_gray, cv2.TM_CCOEFF_NORMED)

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
top_left = max_loc
bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])

In [181]:
cv2.rectangle(background, top_left, bottom_right, (0, 255, 0), 2)

array([[[136, 159, 204],
        [139, 162, 207],
        [143, 166, 211],
        ...,
        [131, 153, 201],
        [127, 150, 198],
        [121, 144, 192]],

       [[154, 178, 220],
        [152, 176, 218],
        [148, 172, 214],
        ...,
        [126, 149, 194],
        [123, 146, 192],
        [117, 142, 186]],

       [[145, 171, 208],
        [144, 170, 207],
        [143, 169, 206],
        ...,
        [127, 150, 188],
        [126, 148, 189],
        [120, 146, 183]],

       ...,

       [[192, 200, 200],
        [191, 199, 199],
        [190, 198, 198],
        ...,
        [175, 181, 180],
        [177, 183, 182],
        [179, 185, 184]],

       [[192, 200, 200],
        [191, 199, 199],
        [190, 198, 198],
        ...,
        [175, 181, 180],
        [177, 183, 182],
        [179, 185, 184]],

       [[192, 200, 200],
        [191, 199, 199],
        [190, 198, 198],
        ...,
        [175, 181, 180],
        [177, 183, 182],
        [179, 185, 184]]

In [369]:
cv2.imshow('Result', background)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('TemplateMatching.jpg', background)

True

$Twinkle,Twinkle, Little, Star$

line

In [456]:
image = cv2.imread('Twinkle, Twinkle, Little Star2.jpg')

gray = image

edges = cv2.Canny(gray,50,150,apertureSize=3)

lines_list =[]
lines = cv2.HoughLinesP(
            edges, # Input edge image
            1, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=100, # Min number of votes for valid line
            minLineLength=800, # Min allowed length of line
            maxLineGap=40 # Max allowed gap between line for joining them
            )
  
# Iterate over points
for points in lines:
      # Extracted points nested in the list
    x1,y1,x2,y2=points[0]
    lines_list.append([(x1,y1),(x2,y2)])

# Define a list to store the filtered lines
filtered_lines = []

def vertical_distance(p1, p2):
    return abs(p1[1] - p2[1])

for points in lines:
    # Extract points nested in the list
    x1, y1, x2, y2 = points[0]
    is_close = False

    for line in filtered_lines:
        y1_existing, y2_existing = line[0][1], line[1][1]
        dist1 = vertical_distance((x1, y1), (x1, y1_existing))
        dist2 = vertical_distance((x2, y2), (x2, y2_existing))

        if dist1 < 20 or dist2 < 20:
            is_close = True
            break
            
    if not is_close:
        filtered_lines.append([(x1, y1), (x2, y2)])
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 1)

        
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

finding notes

In [489]:
template_path = 'B4.jpg'
background_path = 'Twinkle, Twinkle, Little Star2.jpg'

template = cv2.imread(template_path)
background = cv2.imread(background_path)

template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
 

In [495]:
result = cv2.matchTemplate(background_gray, template_gray, cv2.TM_CCOEFF_NORMED)

threshold = 0.75
locations = np.where(result >= threshold)
unique_locations = np.unique(locations)

points = list(zip(*locations[::-1]))

def filter_points(points, min_distance=30):
    filtered_points = []
    for point in points:
        if not filtered_points:
            filtered_points.append(point)
        else:
            distances = cdist([point], filtered_points)
            if np.all(distances > min_distance):
                filtered_points.append(point)
    return filtered_points

filtered_points = filter_points(points)

filtered_locations = (np.array([p[1] for p in filtered_points]), np.array([p[0] for p in filtered_points]))

numbers =[]
numbers2 =[]
num=0
for pt in zip(*filtered_locations[::-1]):
        cv2.rectangle(background, pt , (pt[0] + 60 , pt[1] + 20), (0, 255, 0), 1)
        given_location = pt  
        lines_within_range = []
        for line in filtered_lines:
            distance = (given_location[1] - line[0][1])
            if 0 <= distance <= 150:
                lines_within_range.append(line)
        if len(lines_within_range)==5:
            numbers.append(len(lines_within_range)) 
        elif len(lines_within_range)==3:
            numbers.append(len(lines_within_range))
        elif len(lines_within_range)==2:
            numbers.append(len(lines_within_range))
        elif len(lines_within_range)==4:
            h=0
            for line in lines_within_range:
                if given_location[1] - line[0][1] >= 20 and given_location[1] - line[0][1] <= 30 and h==0:
                    given_location[1] - line[0][1]
                    h = 1
                    numbers.append(41) 
                    break
            if h==0:
                numbers.append(len(lines_within_range))
            

In [465]:
cv2.imshow('Result', background)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite('Twinkle_rec.jpg', background)

In [496]:
line1=[]
line2=[]
line3=[]
line1_num=[]
line2_num=[]
line3_num=[]
i=0
for pt in zip(*filtered_locations[::-1]):
    if  pt[1]<=550:
        line1_num.append(numbers[i])
        line1.append(pt)
    elif pt[1]>550 and pt[1]<=900:
        line2_num.append(numbers[i])
        line2.append(pt)
    elif pt[1]>900 :
        line3_num.append(numbers[i])
        line3.append(pt)
    else:
        continue
    i = i + 1


In [502]:
combined_line1 = list(zip(line1, line1_num))

sorted_line1 = sorted(combined_line1, key=lambda x: x[0][0])

sorted_line1_num = [x[1] for x in sorted_line1]

combined_line2 = list(zip(line2, line2_num))

sorted_line2 = sorted(combined_line2, key=lambda x: x[0][0])

sorted_line2_num = [x[1] for x in sorted_line2]

combined_line3 = list(zip(line3, line3_num))

sorted_line3 = sorted(combined_line3, key=lambda x: x[0][0])

sorted_line3_num = [x[1] for x in sorted_line3]

combined_num = sorted_line1_num  + sorted_line2_num + sorted_line3_num 


In [15]:
fb3=2**(2/12) *  2**2 * 55
fb4=2**(2/12) *  2**3 * 55
fc4=2**(3/12)  *  2**3 * 55
fd4=2**(5/12)  *  2**3 * 55
fg4=2**(10/12)  *  2**3 * 55
fa4=2**(0/12)  *  2**3 * 55
ff4=2**(8/12)  *  2**3 * 55
fe4=2**(7/12)  *  2**3 * 55

freqs = []
for num in combined_num:
    if num == 2:
        freqs.append(fa4)
    elif num == 3:
        freqs.append(fg4)
    elif num == 5:
        freqs.append(fc4)
    elif num == 4:
        freqs.append(fe4)
    elif num == 41:
        freqs.append(ff4)
        
sample_rate = 44100
duration = 4

music = []
i = 0

while i < len(combined_num):
    if combined_num[i] == 5:
        music.append([3, 4, 1])
        i += 1
    elif combined_num[i] == 4:
        music.append([7, 4, 1])
        i += 1
    elif combined_num[i] == 3:
        music.append([10, 4, 1])
        i += 1
    elif combined_num[i] == 2:
        music.append([0, 4, 1])
        i += 1
    elif combined_num[i] == 41:
        music.append([9, 4, 1])
        i += 1
    else:
        i += 1
        
y = get_music(music, sample_rate)
sd.play(y, sample_rate)


NameError: name 'combined_num' is not defined

$Ave\;Maria$

In [309]:
image = cv2.imread('Ave Maria.jpg')

gray = image

edges = cv2.Canny(gray,50,150,apertureSize=3)

lines_list =[]
lines = cv2.HoughLinesP(
            edges, # Input edge image
            1, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=100, # Min number of votes for valid line
            minLineLength=800, # Min allowed length of line
            maxLineGap=40 # Max allowed gap between line for joining them
            )
  
# Iterate over points
for points in lines:
      # Extracted points nested in the list
    x1,y1,x2,y2=points[0]
    lines_list.append([(x1,y1),(x2,y2)])

# Define a list to store the filtered lines
filtered_lines = []

def vertical_distance(p1, p2):
    return abs(p1[1] - p2[1])

for points in lines:
    # Extract points nested in the list
    x1, y1, x2, y2 = points[0]
    is_close = False

    for line in filtered_lines:
        y1_existing, y2_existing = line[0][1], line[1][1]
        dist1 = vertical_distance((x1, y1), (x1, y1_existing))
        dist2 = vertical_distance((x2, y2), (x2, y2_existing))

        if dist1 < 10 or dist2 < 10:
            is_close = True
            break
            
    if not is_close:
        filtered_lines.append([(x1, y1), (x2, y2)])
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 1)

        
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [353]:
template_path1 = 'Ba.jpg'
template_path2 = 'Half.jpg'
template_path3 = 'whole.jpg'
background_path = 'Ave Maria.jpg'

template1 = cv2.imread(template_path1)
template2 = cv2.imread(template_path2)
template3 = cv2.imread(template_path3)
background = cv2.imread(background_path)

template_gray1 = cv2.cvtColor(template1, cv2.COLOR_BGR2GRAY)
template_gray2 = cv2.cvtColor(template2, cv2.COLOR_BGR2GRAY)
template_gray3 = cv2.cvtColor(template3, cv2.COLOR_BGR2GRAY)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
 

qua

In [354]:
result = cv2.matchTemplate(background_gray, template_gray1, cv2.TM_CCOEFF_NORMED)

threshold = 0.8
locations = np.where(result >= threshold)
unique_locations = np.unique(locations)

points = list(zip(*locations[::-1]))

def filter_points(points, min_distance=70):
    filtered_points = []
    for point in points:
        if not filtered_points:
            filtered_points.append(point)
        else:
            distances = cdist([point], filtered_points)
            if np.all(distances > min_distance):
                if not any((np.abs(np.array(point) - np.array(p)) < min_distance).all() for p in filtered_points):
                    filtered_points.append(point)
    return filtered_points


filtered_points1 = filter_points(points)

filtered_locations1 = (np.array([p[1] for p in filtered_points1]), np.array([p[0] for p in filtered_points1]))


numbers=[]

for pt in zip(*filtered_locations1[::-1]):
        cv2.rectangle(background, pt , (pt[0] + 40 , pt[1] + 20), (0, 255, 0), 1)
        given_location = pt  
        lines_within_range = []
        for line in filtered_lines:
            distance = (given_location[1] - line[0][1])
            if 0 <= distance <= 300:
                lines_within_range.append(line)
        if len(lines_within_range)==3:
            numbers.append(31)
            h=0
            for line in lines_within_range:
                if given_location[1] - line[0][1] >= 20 and given_location[1] - line[0][1] <= 40 and h==0:
                    h = 1
                    numbers.append(311) 
                    break
            if h==0:
                numbers.append(31)
        elif len(lines_within_range)==2:
            numbers.append(21)
        elif len(lines_within_range)==4:
            h=0
            for line in lines_within_range:
                if given_location[1] - line[0][1] >= 20 and given_location[1] - line[0][1] <= 40 and h==0:
                    given_location[1] - line[0][1]
                    h = 1
                    numbers.append(411) 
                    break
            if h==0:
                numbers.append(41) 

line111=[]
                
for pt in zip(*filtered_locations1[::-1]):
    line111.append(pt)
                
combined_line111 = list(zip(line111, numbers))

sorted_line111 = sorted(combined_line111, key=lambda x: x[0][0])

sorted_line_num111 = [x[1] for x in sorted_line111]

line112 = sorted(line111, key=lambda x: x[0])

numbers=sorted_line_num111

Half

In [355]:
result = cv2.matchTemplate(background_gray, template_gray2, cv2.TM_CCOEFF_NORMED)

threshold = 0.75
locations = np.where(result >= threshold)
unique_locations = np.unique(locations)

points = list(zip(*locations[::-1]))

filtered_points2 = filter_points(points)

filtered_locations2 = (np.array([p[1] for p in filtered_points2]), np.array([p[0] for p in filtered_points2]))


for pt in zip(*filtered_locations2[::-1]):
        cv2.rectangle(background, pt , (pt[0] + 40 , pt[1] + 20), (255, 0, 0), 1)
        given_location = pt  
        lines_within_range = []
        for line in filtered_lines:
            distance = (given_location[1] - line[0][1])
            if 0 <= distance <= 300:
                lines_within_range.append(line)
        if len(lines_within_range)==3:
            h=0
            for line in lines_within_range:
                if given_location[1] - line[0][1] >= 10 and given_location[1] - line[0][1] <= 20 and h==0:
                    given_location[1] - line[0][1]
                    h = 1
                    numbers.append(312) 
                    break
            if h==0:
                numbers.append(32)
        elif len(lines_within_range)==2:
            numbers.append(22)

whole

In [356]:
result = cv2.matchTemplate(background_gray, template_gray3, cv2.TM_CCOEFF_NORMED)

threshold = 0.75
locations = np.where(result >= threshold)
unique_locations = np.unique(locations)

points = list(zip(*locations[::-1]))

filtered_points3 = filter_points(points)

filtered_locations3 = (np.array([p[1] for p in filtered_points3]), np.array([p[0] for p in filtered_points3]))

for pt in zip(*filtered_locations3[::-1]):
        cv2.rectangle(background, pt , (pt[0] + 50 , pt[1] + 20), (0, 0, 255), 1)
        given_location = pt  
        lines_within_range = []
        for line in filtered_lines:
            distance = (given_location[1] - line[0][1])
            if 0 <= distance <= 300:
                lines_within_range.append(line)
        numbers.append(6)


In [357]:
cv2.imshow('Result', background)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite('Ave_rec.jpg', background)

In [358]:
line=line112
# for pt in line111:
#     line.append(pt)
for pt in zip(*filtered_locations2[::-1]):
    line.append(pt)
for pt in zip(*filtered_locations3[::-1]):
    line.append(pt)

combined_line = list(zip(line, numbers))

sorted_line = sorted(combined_line, key=lambda x: x[0][0])

sorted_line_num = [x[1] for x in sorted_line]
sorted_line_num

[312, 311, 311, 22, 32, 6, 32, 21, 31, 31, 31, 411, 311, 6]

In [359]:
fb4=2**(2/12) *  2**3 * 55
fc4=2**(3/12)  *  2**3 * 55
fd4=2**(5/12)  *  2**3 * 55
fg4=2**(10/12)  *  2**3 * 55
fa4=2**(0/12)  *  2**3 * 55
ff4=2**(8/12)  *  2**3 * 55
fe4=2**(7/12)  *  2**3 * 55

freqs = []
duration = []
music = [] 

for num in sorted_line_num:
    if num == 21:
        freqs.append(fa4)
        duration.append(1)
        music.append([0, 4, 1])
    elif num == 31:
        freqs.append(fg4)
        duration.append(1)
        music.append([10, 4, 1])
    elif num == 311:
        freqs.append(ff4)
        duration.append(1)
        music.append([8, 4, 1])
    elif num == 41:
        freqs.append(fe4)
        duration.append(1)
        music.append([7, 4, 1])
    elif num == 411:
        freqs.append(fd4)
        duration.append(1)
        music.append([6, 4, 1])
    elif num == 6:
        freqs.append(ff4)
        duration.append(4)
        music.append([8, 4, 4])
    elif num == 22:
        freqs.append(fa4)
        duration.append(3)
        music.append([0, 4, 3])
    elif num == 32:
        freqs.append(fg4)
        duration.append(3)
        music.append([10, 4, 3])
    elif num == 312:
        freqs.append(ff4)
        duration.append(3)
        music.append([8, 4, 3])
        
    
    
        
sample_rate = 44100

y = get_music(music, sample_rate)
sd.play(y, sample_rate)